# Counter-Currents Articles

I'm working with a set of about 3,000 short text documents from the Counter-Currents, a white nationalist website. 

I need to perform some preprocessing, then vectorize the documents.

Note that the file with which we're working collects all the records into a single document. Individual cases are delimited with two newlines, hence we split on '\n\n'.

In [1]:
#Imports 

%matplotlib inline
import sys
from   collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import requests
import seaborn as sns
from nltk import word_tokenize, sent_tokenize
from sklearn.datasets import make_blobs
import spacy
from collections import defaultdict
import string 
import pandas as pd
from glob import glob
from scipy import stats
import re
import csv


In [2]:
print(sys.version)

#################################

from datetime import date

today = date.today()
print("Today's date:", today)

3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]
Today's date: 2020-12-09


In [3]:
blogrows = []

with open('blogtext.csv') as f:
    reader = csv.DictReader(f)
    #for row in reader: (if everything worked, this would have worked!)
    done = False
    while not done:
        try:
            row = next(reader)
            if len(row["text"].split())>100:
                blogrows.append(row)
        except Exception as e:
            if str(e) == '':
                done = True

In [4]:
blogtext_df = pd.DataFrame(blogrows)
blogtext_df['text'] = blogtext_df['text'].str.lower()
blogtext_df['publication'] = 'blogtext'
blogtext_df.head()

,id,gender,age,topic,sign,date,text,publication
0,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde...,blogtext
1,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",i had an interesting conversation...,blogtext
2,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",somehow coca-cola has a way of su...,blogtext
3,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","if anything, korea is a country o...",blogtext
4,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",take a read of this news article ...,blogtext


In [5]:
len(blogtext_df.values)

362219

In [6]:
newsrows = []

outlets =['CNN', 'Fox']

with open('all-the-news-2-1.csv') as f:
    reader = csv.DictReader(f)
    #for row in reader: (if everything worked, this would have worked!)
    done = False
    while not done:
        try:
            row = next(reader)
            if len(row["article"].split())>100 and (outlets[0] in row["publication"] or outlets[1] in row["publication"]):
                newsrows.append(row)
        except Exception as e:
            if str(e) == '':
                done = True

In [7]:
newstext_df = pd.DataFrame(newsrows)
newstext_df['text'] = newstext_df['article'].str.lower()
newstext_df.head()

,,Unnamed: 0,date,year,month,day,author,title,article,url,section,publication,text
0,834313,834313,2016-01-01 00:00:00,2016,1.0,1,By David Bianculli,"Wayne Rogers, and why we needed M*A*S*H",David Bianculli is founder and editor of TVWor...,https://www.cnn.com/2016/01/01/opinions/biancu...,opinions,CNN,david bianculli is founder and editor of tvwor...
1,834314,834314,2016-01-01 00:00:00,2016,1.0,1,Ben Brumfield,South Illinois towns evacuate as water tops levee,Are you within sight of floodwater? Share your...,https://www.cnn.com/2016/01/01/us/weather-rive...,us,CNN,are you within sight of floodwater? share your...
2,834317,834317,2016-01-01 00:00:00,2016,1.0,1,"By Jason Hanna and Michael Schwartz, CNN","Tel Aviv pub shooting leaves 2 dead, 7 injured",(CNN)A gunman on Friday sprayed bullets from ...,https://www.cnn.com/2016/01/01/middleeast/tel-...,middleeast,CNN,(cnn)a gunman on friday sprayed bullets from ...
3,834318,834318,2016-01-01 00:00:00,2016,1.0,1,Harmeet S. Singh,Air India flight turns back over rat scare,(CNN)Snakes on a plane? How about a rat? An A...,https://www.cnn.com/2016/01/01/aviation/india-...,aviation,CNN,(cnn)snakes on a plane? how about a rat? an a...
4,834319,834319,2016-01-01 00:00:00,2016,1.0,1,Evan Perez,Justice Department investigating Blue Bell Cre...,(CNN)Blue Bell Creameries is under criminal i...,https://www.cnn.com/2016/01/01/health/blue-bel...,health,CNN,(cnn)blue bell creameries is under criminal i...


In [8]:
len(newstext_df.values)

143584

In [9]:
bb_rows = []


with open('newsbb.csv') as f:
    reader = csv.DictReader(f)
    #for row in reader: (if everything worked, this would have worked!)
    done = False
    while not done:
        try:
            row = next(reader)
            if len(row["content"].split())>100 and ('Breitbart' in row["publication"]):
                bb_rows.append(row)
        except Exception as e:
            if str(e) == '':
                done = True

In [10]:
breit_df = pd.DataFrame(bb_rows)
breit_df['text'] = breit_df['content'].str.lower()
display(breit_df.head())
print(len(breit_df.values))

,,id,title,publication,author,date,year,month,url,content,text
0,7803,26539,CNN’s Zeleny: ’Hard to Imagine’ Obama Would Ha...,Breitbart,Ian Hanchett,2017-01-17,2017.0,1.0,,On Tuesday’s broadcast of CNN’s “Situation Roo...,on tuesday’s broadcast of cnn’s “situation roo...
1,7804,26540,American Students on Spring Break Chant ’Build...,Breitbart,Katherine Rodriguez,2017-03-21,2017.0,3.0,,A group of American spring break revelers repo...,a group of american spring break revelers repo...
2,7805,26541,Surge in ’Honour Crimes’ and Forced Marriages ...,Breitbart,Liam Deacon,2017-04-08,2017.0,4.0,,“honour crimes” have risen by 40 per cent in...,“honour crimes” have risen by 40 per cent in...
3,7806,26542,MILO Announces New Media Venture - Breitbart,Breitbart,Lucas Nolan,2017-04-28,2017.0,4.0,,Former Breitbart Senior Editor MILO has announ...,former breitbart senior editor milo has announ...
4,7807,26543,Jared Kushner at Center of Media Spotlight on ...,Breitbart,Penny Starr,2017-05-27,2017.0,5.0,,The focus of the continuous media reports of a...,the focus of the continuous media reports of a...


22855


In [11]:
CCp1 = pd.read_csv('CounterCurrentsDatapulledtttt_2020-11-30.csv', index_col=0)

In [12]:
CCp1.head()

,url,author,ext_title,content,pubdate,pubbed_in,other_tags
"El Factor Moral, Parte 1",https://counter-currents.com/2015/05/el-factor...,Greg Johnson,"Greg Johnson, ""El Factor Moral,"" Parte 1 | Cou...","\nFélicien Rops, “Pornokratès,” 1878\n2,361 wo...","May 27, 2015",This entry was posted in North American New Right,"['articles', 'cynicism', 'ethics', 'Greg Johns..."
Good Kill,https://counter-currents.com/2015/05/good-kill/,Trevor Lynch,Trevor Lynch reviews Good Kill | Counter-Currents,\n870 words\nGood Kill is an OK movie starring...,"May 27, 2015",This entry was posted in North American New Right,"['Afghanistan', 'Gattaca', 'movie reviews', 'N..."
"Remembering Louis-Ferdinand Céline: May 27, 1894–July 1, 1961",https://counter-currents.com/2015/05/rememberi...,Greg Johnson,"Greg Johnson, Remembering Remembering Louis-Fe...",\n162 words\nLouis-Ferdinand Céline was the pe...,"May 27, 2015",This entry was posted in North American New Right,"['commemorations', 'fascism', 'Greg Johnson', ..."
Céline,https://counter-currents.com/2015/05/celine/,Leo Yankevich,"Leo Yankevich, ""Céline"" | Counter-Currents",\n64 words\nThree pamphlets in which he spared...,"May 26, 2015",This entry was posted in North American New Right,"['France', 'Leo Yankevich', 'Louis-Ferdinand C..."
The Enduring Reputation of Louis-Ferdinand Céline,https://counter-currents.com/2015/05/the-endur...,Margot Metroland,"Margot Metroland, ""The Enduring Reputation of ...","\n1,282 words\n“The white people invented the ...","May 26, 2015",This entry was posted in North American New Right,"['articles', 'collaborationism', 'France', 'Jo..."


In [13]:
CCp2 = pd.read_csv('CounterCurrentsDatapulled_2020-11-29_1.csv', index_col=0)

In [14]:
CCp2.head()

,url,author,ext_title,content,pubdate,pubbed_in,other_tags
The Counter-Currents 2020 Fundraiser,https://counter-currents.com/2020/05/all-hands...,Greg Johnson,The Counter-Currents 2020 Fundraiser All Hands...,\n804 words\nThere’s never a good time for a p...,"May 25, 2020",This entry was posted in North American New Right,['2020 fundraiser']
Sun & Steel: The Tatenokai Phenomenon in Brief,https://counter-currents.com/2020/11/sun-and-s...,Turan,The Tatenokai Phenomenon in Brief | Counter-Cu...,"\n1,919 words\nIn post-1945 Japan — as in most...","November 27, 2020",This entry was posted in North American New Right,"['articles', 'fascism', 'Japan', 'liberalism',..."
Fox News & the Boomer Question,https://counter-currents.com/2020/11/fox-news-...,Alex Graham,Fox News & the Boomer Question | Counter-Currents,"\n1,064 words\nFox News’ coverage of the 2020 ...","November 27, 2020",This entry was posted in North American New Right,"['Alex Graham', 'alternative media', 'articles..."
Black Friday Special It’s Time to STOP Shopping for Christmas,https://counter-currents.com/2019/11/its-time-...,Greg Johnson,Black Friday Special It's Time to STOP Shoppin...,"\n1,031 words / 5:45\nAudio version: To listen...","November 28, 2019",This entry was posted in North American New Right,"['articles', 'Christmas', 'commercial society'..."
Video of the Day: Poetry With a Splash of Blood,https://counter-currents.com/2020/11/poetry-wi...,Video of the Day,Livestream: Poetry With a Splash of Blood | Co...,\n152 words\nIn the latest episode of Guide to...,"November 27, 2020",This entry was posted in North American New Right,"['Frodi Midjord', 'Greg Johnson', 'Guide to Ku..."


In [15]:
combo = [CCp1, CCp2]
CC_data = pd.concat(combo)
CC_data['publication']= 'CounterCurrents'
CC_data['text']= CC_data['content'].str.lower()

In [16]:
print(len(CC_data['url'].values))

3606


In [89]:
blogtext_df.to_pickle(r'blogtext_data.pkl')
newstext_df.to_pickle(r'newstext_data.pkl')
breit_df.to_pickle(r'breit_data.pkl')
CC_data.to_pickle(r'CC_data.pkl')

TypeError: can only concatenate str (not "datetime.date") to str